In [783]:
# 1. import packages
import pandas as pd
from datetime import datetime
from pprint import pprint
from sklearn import datasets
import matplotlib.pyplot as plt
import seaborn as sns

In [784]:
# 2. changes settings (optional)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 100)
pd.set_option('display.precision', 3)

In [785]:
# 3. define function that will gather data from csv files, join together into a single DataFrame
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        #parse_dates-list of lists. e.g. If [[1, 3]] -> combine columns 1 and 3 and parse as a single date column.
        dfs.append(pd.read_csv(file_url, parse_dates=[['DATE','TIME']], keep_date_col=True))
    return pd.concat(dfs,ignore_index = True)

In [786]:
# 4. define csv files (Nov 2019 - Feb 2020 data); call function and save output in variable
week_nums = [191102, 191109, 191116,191123,191130,191207,191214,191221,191228,200104,200111,200118,200125,200201,200208,200215,200222,200229]
turnstiles_df = get_data(week_nums)

In [787]:
# 5. clear whitespace, reindex the columns and then view the columns
turnstiles_df.columns = [column.strip() for column in turnstiles_df.columns]
turnstiles_df.reindex()
turnstiles_df.columns

Index(['DATE_TIME', 'C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION',
       'DATE', 'TIME', 'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

### There is no need to run the above code again ###

In [788]:
# 6. add any new columns and/or standardize columns
turnstiles_df['NEW_LINE'] = turnstiles_df.LINENAME.apply(lambda x: ''.join(sorted(x)))
turnstiles_df["STATION_LINE"] = turnstiles_df.STATION + " " + turnstiles_df.NEW_LINE
turnstiles_df["WEEKDAY"] = turnstiles_df['DATE_TIME'].dt.day_name()

In [744]:
# 7. get a feel for the data by viewing number of unique elements in each column, this steps helps in indentifying
#    which columns to take a close look at in the process of cleaning the data
nunique_d = {}
for x in turnstiles_df.columns:
    nunique_d[x] = turnstiles_df[x].nunique()
pprint(nunique_d)

{'C/A': 748,
 'DATE': 126,
 'DATE_TIME': 244815,
 'DESC': 1,
 'DIVISION': 6,
 'ENTRIES': 2521301,
 'EXITS': 2319072,
 'LINENAME': 114,
 'NEW_LINE': 95,
 'SCP': 225,
 'STATION': 378,
 'STATION_LINE': 468,
 'TIME': 70117,
 'UNIT': 469,
 'WEEKDAY': 7}


In [745]:
# 8. *Q* There are more unique C/A than UNITS which seems odd. A C/A (control area) is the booth where the 
#    station master sits so why are there more distinct C/A's than UNITS?

#Sense check
(turnstiles_df.groupby(['C/A','STATION_LINE',"UNIT"]).ENTRIES.count()
 .reset_index().head(50).sort_values(["UNIT"], ascending=(False)).head(100))

#    *A* It seems UNITS are a collection of turnstiles almost synonimous with station. 1 station - 1 Unit
#              On the other hand there are usually 1-4 C/A per station depending on size.

,C/A,STATION_LINE,UNIT,ENTRIES
24,A046,CANAL ST 6JNQRWZ,R463,14370
23,A043,CANAL ST 6JNQRWZ,R462,8356
47,B010,BOTANIC GARDEN 2345S,R412,3036
46,B009,PARK PLACE S,R411,3780
35,A064,BOWERY JZ,R311,2269
30,A054,RECTOR ST NRW,R227,4529
31,A055,RECTOR ST NRW,R227,3770
49,B013,PROSPECT PARK BQS,R196,3767
48,B012,PROSPECT PARK BQS,R196,2262
45,B004,7 AV BQ,R171,3017


In [780]:
##### General Thoughts #####
#A few preliminary thoughts after having looked at head, tail, sample, unique data etc.
# 1. STATION_LINE is a better identifier for particular stations, so it is better to use that column for grouping
#    rather than just grouping STATION or LINE data on its own
# 2. DATE is a better identifier for grouping than DATETIME at least to achieve the MVP. DATETIME can be used to 
#    to divide a day into several timeframes but thats a Day 2 item
# 3. Before we can group on STATION_LINE and DATE, all the adjustments and data cleaning needs to be done at a
#    granular level (i.e we need to initially group the data in a way where turnstile unit info. is shown rather
#    than STATION_LINE info. as a whole)
# 4. There are 476 unique STATION_LINE in the November Dataset, and we need to find the most popular ones...

#Below code returns the names of 468 unique STATION_LINEs, while I haven't gone through the list for accuracy
#it seems complete (verified using google "how many subway stops in NYC?"). No further analysis needed for 
#completeness, accuracy, consistency are needed on STATION LINEs at this time for two reasons. The first is
#that we a reasonably certain of the most popular stations through general knowledge. The second is that we are
#looking for the MVP and cannot be spending too much time on any one attribute.
##### General Thoughts #####

for x in sorted(turnstiles_df.STATION_LINE.unique()):
    print(x)

1 AV L
103 ST 1
103 ST 6
103 ST BC
103 ST-CORONA 7
104 ST A
104 ST JZ
110 ST 6
111 ST 7
111 ST A
111 ST J
116 ST 23
116 ST 6
116 ST BC
116 ST-COLUMBIA 1
121 ST JZ
125 ST 1
125 ST 23
125 ST 456
125 ST ABCD
135 ST 23
135 ST BC
137 ST CITY COL 1
138/GRAND CONC 45
14 ST 123FLM
14 ST ACEL
14 ST-UNION SQ 456LNQRW
145 ST 1
145 ST 3
145 ST ABCD
149/GRAND CONC 245
14TH STREET 1
15 ST-PROSPECT FG
155 ST BD
155 ST C
157 ST 1
161/YANKEE STAD 4BD
163 ST-AMSTERDM C
167 ST 4
167 ST BD
168 ST 1AC
169 ST F
170 ST 4
170 ST BD
174 ST 25
174-175 STS BD
175 ST A
176 ST 4
18 AV D
18 AV F
18 AV N
18 ST 1
181 ST 1
181 ST A
182-183 STS BD
183 ST 4
190 ST A
191 ST 1
2 AV F
20 AV D
20 AV N
207 ST 1
21 ST G
21 ST-QNSBRIDGE F
215 ST 1
219 ST 25
225 ST 25
23 ST 1
23 ST 6
23 ST CE
23 ST FM
23 ST NRW
231 ST 1
233 ST 25
238 ST 1
25 AV D
25 ST R
28 ST 1
28 ST 6
28 ST NRW
3 AV 138 ST 6
3 AV L
3 AV-149 ST 25
30 AV NQW
33 ST 6
33 ST-RAWSON ST 7
34 ST-HERALD SQ BDFMNQRW
34 ST-HUDSON YD 7
34 ST-PENN STA 123ACE
36 AV NQW
36 

In [670]:
#ASK TEAM#
# 9. *Q* What to do for unique station lines that are very similar?
#    A mask is similar to a where clause in SQL and will basically allow us to filter the data so that we can 
#    look at the subset of the overall population to identify discrepancies and eliminate them.
#    After having looked at the unique station list, it seems that the below two stations are very similarly 
#    named. Are they different or the same, should we be combining these records? If so, how?
mask = (turnstiles_df["STATION_LINE"].isin(['TIMES SQ-42 ST 1237ACENQRSW','TIMES SQ-42 ST 1237ACENQRS']))
(turnstiles_df[mask].groupby(["STATION_LINE","C/A","UNIT","SCP"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False))


#    *A* It was decided that both TIME SQ are the same and PORT AUTH is also Time Sq

,STATION_LINE,C/A,UNIT,SCP,ENTRIES
0,TIMES SQ-42 ST 1237ACENQRS,R147,R033,04-00-00,764
2,TIMES SQ-42 ST 1237ACENQRS,R147,R033,04-00-02,764
7,TIMES SQ-42 ST 1237ACENQRS,R147,R033,04-05-00,764
8,TIMES SQ-42 ST 1237ACENQRS,R147,R033,04-05-01,764
3,TIMES SQ-42 ST 1237ACENQRS,R147,R033,04-00-03,763
4,TIMES SQ-42 ST 1237ACENQRS,R147,R033,04-00-04,763
1,TIMES SQ-42 ST 1237ACENQRS,R147,R033,04-00-01,762
5,TIMES SQ-42 ST 1237ACENQRS,R147,R033,04-00-05,761
34,TIMES SQ-42 ST 1237ACENQRSW,R148,R033,01-00-01,758
38,TIMES SQ-42 ST 1237ACENQRSW,R148,R033,01-03-03,756


In [791]:
# 10. Replace station names that are duplicates to a single station name so that when we 
turnstiles_df['STATION_LINE'].replace(to_replace =['34 ST-PENN STA 123','34 ST-PENN STA ACE'], value ='34 ST-PENN STA 123ACE',inplace=True) 
turnstiles_df['STATION_LINE'].replace(to_replace =['TIMES SQ-42 ST 1237ACENQRS','42 ST-PORT AUTH 1237ACENQRSW','42 ST-PORT AUTH 1237ACEGNRSW','TIMES SQ-42 ST 1237ACENQRSW'], value ='TIMES SQ 42 ST-1237ACENQRSW',inplace=True)


In [792]:
for x in sorted(turnstiles_df.STATION_LINE.unique()):
    print(x)

1 AV L
103 ST 1
103 ST 6
103 ST BC
103 ST-CORONA 7
104 ST A
104 ST JZ
110 ST 6
111 ST 7
111 ST A
111 ST J
116 ST 23
116 ST 6
116 ST BC
116 ST-COLUMBIA 1
121 ST JZ
125 ST 1
125 ST 23
125 ST 456
125 ST ABCD
135 ST 23
135 ST BC
137 ST CITY COL 1
138/GRAND CONC 45
14 ST 123FLM
14 ST ACEL
14 ST-UNION SQ 456LNQRW
145 ST 1
145 ST 3
145 ST ABCD
149/GRAND CONC 245
14TH STREET 1
15 ST-PROSPECT FG
155 ST BD
155 ST C
157 ST 1
161/YANKEE STAD 4BD
163 ST-AMSTERDM C
167 ST 4
167 ST BD
168 ST 1AC
169 ST F
170 ST 4
170 ST BD
174 ST 25
174-175 STS BD
175 ST A
176 ST 4
18 AV D
18 AV F
18 AV N
18 ST 1
181 ST 1
181 ST A
182-183 STS BD
183 ST 4
190 ST A
191 ST 1
2 AV F
20 AV D
20 AV N
207 ST 1
21 ST G
21 ST-QNSBRIDGE F
215 ST 1
219 ST 25
225 ST 25
23 ST 1
23 ST 6
23 ST CE
23 ST FM
23 ST NRW
231 ST 1
233 ST 25
238 ST 1
25 AV D
25 ST R
28 ST 1
28 ST 6
28 ST NRW
3 AV 138 ST 6
3 AV L
3 AV-149 ST 25
30 AV NQW
33 ST 6
33 ST-RAWSON ST 7
34 ST-HERALD SQ BDFMNQRW
34 ST-HUDSON YD 7
34 ST-PENN STA 123ACE
36 AV NQW
36 

In [793]:
##### General Thoughts #####
#The following code shows that the count of rows are different for each station. A station with more UNITS 
#and SUBUNITS will have more SUBUNITS being tracked..as seen below
#Grand Centeral has the most amount of data entries as it has the most turnstiles that are reporting back
#If we divide row count of each station by the turnstiles in each station, the quotient should represent the total
#clock-ins by each turnstile. We we then divde that by the number of days, it will return the daily clock-ins
##### General Thoughts #####
(turnstiles_df
 .groupby(["STATION_LINE"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False))

,STATION_LINE,ENTRIES
434,TIMES SQ 42 ST-1237ACENQRSW,76266
88,34 ST-PENN STA 123ACE,75520
291,FULTON ST 2345ACJZ,67249
300,GRD CNTRL-42 ST 4567S,55966
86,34 ST-HERALD SQ BDFMNQRW,41770
...,...,...
186,BAYCHESTER AV 5,2256
215,BRONX PARK EAST 25,2255
396,PELHAM PKWY 5,2252
430,SUTTER AV L,1512


In [794]:
# view df
turnstiles_df.head(10)

,DATE_TIME,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,NEW_LINE,STATION_LINE,WEEKDAY
0,2019-10-26 00:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,00:00:00,REGULAR,7247322,2455491,456NQRW,59 ST 456NQRW,Saturday
1,2019-10-26 04:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,04:00:00,REGULAR,7247336,2455499,456NQRW,59 ST 456NQRW,Saturday
2,2019-10-26 08:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,08:00:00,REGULAR,7247351,2455532,456NQRW,59 ST 456NQRW,Saturday
3,2019-10-26 12:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,12:00:00,REGULAR,7247463,2455623,456NQRW,59 ST 456NQRW,Saturday
4,2019-10-26 16:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,16:00:00,REGULAR,7247755,2455679,456NQRW,59 ST 456NQRW,Saturday
5,2019-10-26 20:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,20:00:00,REGULAR,7248159,2455733,456NQRW,59 ST 456NQRW,Saturday
6,2019-10-27 00:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/27/2019,00:00:00,REGULAR,7248310,2455757,456NQRW,59 ST 456NQRW,Sunday
7,2019-10-27 04:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/27/2019,04:00:00,REGULAR,7248339,2455765,456NQRW,59 ST 456NQRW,Sunday
8,2019-10-27 08:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/27/2019,08:00:00,REGULAR,7248356,2455782,456NQRW,59 ST 456NQRW,Sunday
9,2019-10-27 12:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/27/2019,12:00:00,REGULAR,7248418,2455860,456NQRW,59 ST 456NQRW,Sunday


In [770]:
# 11. Sanity Check to verify that "STATIONLINE","UNIT", "C/A", "SUBUNIT","DATE_TIME" is unique
(turnstiles_df
 .groupby(["STATION_LINE","UNIT","C/A", "SCP", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values(["ENTRIES"], ascending=(False))).head(10)

,STATION_LINE,UNIT,C/A,SCP,DATE_TIME,ENTRIES
0,1 AV L,R248,H007,00-00-00,2019-10-26 00:00:00,1
2461042,GUN HILL RD 25,R364,R331,00-05-01,2019-11-16 20:00:00,1
2461072,GUN HILL RD 25,R364,R331,00-05-01,2019-11-21 20:00:00,1
2461073,GUN HILL RD 25,R364,R331,00-05-01,2019-11-22 00:00:00,1
2461074,GUN HILL RD 25,R364,R331,00-05-01,2019-11-22 04:00:00,1
2461075,GUN HILL RD 25,R364,R331,00-05-01,2019-11-22 08:00:00,1
2461076,GUN HILL RD 25,R364,R331,00-05-01,2019-11-22 12:00:00,1
2461077,GUN HILL RD 25,R364,R331,00-05-01,2019-11-22 16:00:00,1
2461078,GUN HILL RD 25,R364,R331,00-05-01,2019-11-22 20:00:00,1
2461079,GUN HILL RD 25,R364,R331,00-05-01,2019-11-23 00:00:00,1


In [795]:
# 13. There are duplicate entries in the data due to DESC column, it has "RECOVR AUD" which we should remove
turnstiles_df = turnstiles_df[~turnstiles_df['DESC'].isin(["RECOVR AUD"])]
turnstiles_df.reindex()

,DATE_TIME,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,NEW_LINE,STATION_LINE,WEEKDAY
0,2019-10-26 00:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,00:00:00,REGULAR,7247322,2455491,456NQRW,59 ST 456NQRW,Saturday
1,2019-10-26 04:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,04:00:00,REGULAR,7247336,2455499,456NQRW,59 ST 456NQRW,Saturday
2,2019-10-26 08:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,08:00:00,REGULAR,7247351,2455532,456NQRW,59 ST 456NQRW,Saturday
3,2019-10-26 12:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,12:00:00,REGULAR,7247463,2455623,456NQRW,59 ST 456NQRW,Saturday
4,2019-10-26 16:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,16:00:00,REGULAR,7247755,2455679,456NQRW,59 ST 456NQRW,Saturday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3708423,2020-02-28 04:00:00,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,02/28/2020,04:00:00,REGULAR,5554,504,R,RIT-ROOSEVELT R,Friday
3708424,2020-02-28 08:00:00,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,02/28/2020,08:00:00,REGULAR,5554,504,R,RIT-ROOSEVELT R,Friday
3708425,2020-02-28 12:00:00,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,02/28/2020,12:00:00,REGULAR,5554,504,R,RIT-ROOSEVELT R,Friday
3708426,2020-02-28 16:00:00,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,02/28/2020,16:00:00,REGULAR,5554,504,R,RIT-ROOSEVELT R,Friday


In [752]:
# view df
turnstiles_df.head()

,DATE_TIME,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,NEW_LINE,STATION_LINE,WEEKDAY
0,2019-10-26 00:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-10-26,00:00:00,REGULAR,7247322,2455491,456NQRW,59 ST 456NQRW,Saturday
1,2019-10-26 04:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-10-26,04:00:00,REGULAR,7247336,2455499,456NQRW,59 ST 456NQRW,Saturday
2,2019-10-26 08:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-10-26,08:00:00,REGULAR,7247351,2455532,456NQRW,59 ST 456NQRW,Saturday
3,2019-10-26 12:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-10-26,12:00:00,REGULAR,7247463,2455623,456NQRW,59 ST 456NQRW,Saturday
4,2019-10-26 16:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-10-26,16:00:00,REGULAR,7247755,2455679,456NQRW,59 ST 456NQRW,Saturday


In [796]:
# 14. Now that our data is clean enough, we can start grouping DataFrame to get the daily numbers for each station
# Below we are getting the first entry for any given date at any given turnstile
turnstiles_daily = (turnstiles_df
                        .groupby(["STATION_LINE","UNIT","C/A", "SCP", "DATE"],as_index=False)
                        .ENTRIES.first())

In [797]:
# 15. For the very first date the previous date was not available, therefore we shift everything by one 
turnstiles_daily[["PREV_DATE", "PREV_ENTRIES"]] = (turnstiles_daily
                                                       .groupby(["STATION_LINE","UNIT","C/A", "SCP"])["DATE", "ENTRIES"]
                                                       .apply(lambda grp: grp.shift(1)))

In [755]:
# view df
turnstiles_daily.head(3)

,STATION_LINE,UNIT,C/A,SCP,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES
0,1 AV L,R248,H007,00-00-00,2019-10-26,15099179,NaN,nan
1,1 AV L,R248,H007,00-00-00,2019-10-27,15101784,2019-10-26,"15,099,179.0"
2,1 AV L,R248,H007,00-00-00,2019-10-28,15102950,2019-10-27,"15,101,784.0"


In [798]:
# 15. Drop the rows for the earliest date in the df
turnstiles_daily.dropna(subset=["PREV_DATE"], axis=0, inplace=True)

In [800]:
# 16. Checking to see if entries is less than previous entries
turnstiles_daily[turnstiles_daily["ENTRIES"] < turnstiles_daily["PREV_ENTRIES"]].head()

,STATION_LINE,UNIT,C/A,SCP,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES
53,1 AV L,R248,H007,00-00-00,10/26/2019,15099179,02/28/2020,"15,455,022.0"
166,1 AV L,R248,H007,00-00-01,10/26/2019,60718358,02/15/2020,"61,128,727.0"
279,1 AV L,R248,H007,00-03-00,10/26/2019,370757608,02/15/2020,"370,864,076.0"
392,1 AV L,R248,H007,00-03-01,10/26/2019,2524031,02/15/2020,"2,605,071.0"
505,1 AV L,R248,H007,00-03-02,10/26/2019,6505570,02/15/2020,"6,632,469.0"


In [801]:
# 17. define a function that will get daily counts and run logic for unique cases such as entries decreasing
# day over day and the turnstile resetting day over day
def get_daily_counts(row, max_counter):
    counter = row["ENTRIES"] - row["PREV_ENTRIES"]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        # Maybe counter was reset to 0? 
        print(row["ENTRIES"], row["PREV_ENTRIES"])
        counter = min(row["ENTRIES"], row["PREV_ENTRIES"])
    if counter > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        return 0
    return counter

# If counter is > 50000, then the counter might have been reset.  
# Just set it to zero as different counters have different cycle limits
# It'd probably be a good idea to use a number even significantly smaller than 1 million as the limit!
turnstiles_daily["DAILY_ENTRIES"] = turnstiles_daily.apply(get_daily_counts, axis=1, max_counter=50000)

15099179 15455022.0
60718358 61128727.0
370757608 370864076.0
2524031 2605071.0
6505570 6632469.0
25 77409.0
32 117788.0
29 192736.0
2376276 2695702.0
13654074 123906.0
11 13741429.0
16690109 16942120.0
8466082 8596834.0
12783672 12946971.0
11708838 11934512.0
296357 623504.0
4639786 4902534.0
15081899 15344064.0
831192 995546.0
2042316 2194356.0
25691802 25891106.0
14460580 14700780.0
11006281 11173114.0
5106871 5193184.0
15857408 16063716.0
14581044 14767329.0
2548384 2741454.0
1541343 1800801.0
6181312 6580409.0
12990037 13336504.0
11486640 11772815.0
4195358 4354210.0
4813742 4984692.0
314128 381915.0
15601 101209.0
14424 99770.0
4832609 4961171.0
5561431 5685778.0
5580823 5672090.0
3886503 4175925.0
5168806 5424394.0
14241280 14476237.0
17203453 17573065.0
16037514 16392619.0
12304277 12583854.0
3598237 3876583.0
4693684 4943251.0
2379032 2443545.0
863958 5013389.0
4982314 877275.0
3801860 3859853.0
6912484 6999076.0
4526363 4748852.0
1189047 1383414.0
8187787 8328977.0
629945 766

334806 540772.0
5343763 5490995.0
4687833 4858733.0
104386359 104600479.0
42749 338588.0
5461290 5756590.0
11217375 11437472.0
19737774 269776.0
2292 19783215.0
4050948 4342908.0
2456241 2626210.0
2786780 2935319.0
5231854 5375139.0
3783393 3947762.0
270532634 535694.0
505193 270547467.0
6862977 7109266.0
6804490 189859.0
16 6919384.0
6570478 6873033.0
722638 1027893.0
8132411 8453472.0
582379 833815.0
238384483 238586486.0
675167265 675435528.0
1824820619 1824553842.0
339400781 339655695.0
580262 839936.0
453620 652248.0
159826 285181.0
7101415 7245791.0
5325992 5442935.0
6661181 6808596.0
10163164 10382986.0
11319181 11656020.0
8838999 9066791.0
85974447 86115523.0
1565140 1813234.0
4835160 5013482.0
6706161 6843745.0
2870570 2928765.0
4151761 4235895.0
290955 106458.0
1043 334720.0
8372170 8629009.0
13632416 13816717.0
15782265 15986315.0
11066507 11370580.0
14 462037.0
90080 8377.0
393269 96316.0
458861 393320.0
458761 106666.0
131109 458764.0
458752 131215.0
131401 458752.0
589865

9421909 9638915.0
4932668 5129252.0
6482172 6635311.0
1936935 2060234.0
5198050 84403.0
1298 5249104.0
5197001 5348223.0
98219 272380.0
1112525 1346007.0
11860157 12041438.0
709 1736646.0
1659815 25166.0
2297270 2421965.0
5552351 5675373.0
5813379 5903874.0
5188399 5270440.0
6974338 7110447.0
1714961 1862432.0
10340196 10502236.0
14009143 14267900.0
335442861 335686395.0
10 11957681.0
11822795 55290.0
5392647 5622262.0
3953672 4305060.0
10275934 10510585.0
205714 345526.0
5058395 5132340.0
70692 141430.0
1050657 1108391.0
722 4895019.0
4827307 31403.0
11490991 11704277.0
862803 976726.0
5940581 6069917.0
5472053 5556668.0
1338592 1475286.0
1613553 1694494.0
1257152 1386278.0
31144547 31299017.0
4242446 4337568.0
7176108 7265731.0
4806872 4912955.0
5871369 6015856.0
13550266 13733101.0
531876 596439.0
661598 715544.0
5464913 5530597.0
1322692 1450683.0
569360885 569464559.0
267897 370179.0
8126557 8225856.0
52086 148123.0
1009341 1105485.0
8704105 8825910.0
7744675 7855223.0
3986709 420

1451996 1595094.0
6602136 6761576.0
9631730 9857654.0
15419517 15611413.0
3531764 3776707.0
22712020 23019317.0
2880809 3179687.0
1952783 2143982.0
1768883 1983113.0
6042516 6316911.0
959 7066197.0
6919613 97304.0
6537330 6646208.0
4515809 4639890.0
3925297 4122049.0
11840909 12129246.0
1754914 1821284.0
1563620 1676416.0
505589 607885.0
3165459 3267314.0
691759 802219.0
925831 1043586.0
3925052 4037268.0
1376536 1549750.0
9102431 9376418.0
10407399 10783099.0
1867180 2273524.0
22306352 22735654.0
23498458 23788763.0
24255053 24487557.0
17620170 17812855.0
19201109 19482218.0
3547824 3886262.0
645393285 645822328.0
2675017 2855415.0
90619 266442.0
6056900 6235335.0
655036 717236.0
8135602 8261539.0
11601222 11854565.0
1961895 2208741.0
2487854 2786518.0
2587686 2897814.0
2440308 2735070.0
1493771 1675428.0
1041247 1176495.0
720167 818147.0
478249 544534.0
396276 453188.0
435896 493988.0
756711 845652.0
1289390 1451036.0
849 100946.0
1 122660.0
232312 55399.0
2068 260951.0
209585 143759

1582905 1729489.0
2600522 2832118.0
3024475 3137815.0
11497517 11691489.0
369792 536444.0
303479989 303571271.0
4500929 4713765.0
52022024 52105240.0
419273 608021.0
11697502 11830143.0
2115969039 2116066893.0
478197 569521.0
2267775 2397890.0
8492182 8727815.0
3901862 4032217.0
3511285 102426.0
18 3511766.0
9729368 9825287.0
11990051 12132417.0
8442830 8664887.0
14047230 14431860.0
1158882163 1159035620.0
670794 775303.0
7760283 7848483.0
6642572 6723341.0
6144211 6250124.0
13400313 13571634.0
524549 118146212.0
118100625 536287.0
12180529 12235103.0
4807734 4878528.0
3 69159701.0
69010026 54709.0
102486894 102648530.0
7317606 7382958.0
5624506 5742210.0
38 9500468.0
9424918 27983.0
13777021 13897563.0
127 8437383.0
8362503 32818.0
10932636 11020658.0
28502 102926.0
7772532 7979324.0
857203 1000773.0
10146338 10241548.0
7760829 7839955.0
353286718 353354082.0
4723938 4842194.0
11699 122754.0
10 126195.0
29321 277613.0
6467129 6572225.0
3224083 3413940.0
583332 738991.0
6033806 6156674

118735918 118835216.0
4188200 4244832.0
783826 874436.0
4044044 4172595.0
19537675 19726415.0
14429680 14601086.0
7787211 7929298.0
569442710 569525997.0
135401 324415.0
4764628 4936619.0
235294243 235351038.0
152377 249634.0
135930 203813.0
41577 100134.0
1526474 1580522.0
2621664 2717362.0
1835582 1949903.0
1149253 1224872.0
1653444 1754086.0
993797 1055841.0
1070311 1144019.0
6465181 6613324.0
5424395 5568584.0
4391619 4675615.0
2219698 2283246.0
5501215 5828831.0
3314153 3532431.0
2173783 192278.0
163 2209769.0
7180372 7465097.0
14128375 14354782.0
14755066 15027353.0
1529321 1597179.0
10772332 11006503.0
4033699 4214906.0
11433819 11588289.0
4101263 4336054.0
19847888 20262738.0
1766053 2139191.0
239135364 239473339.0
4440282 4722142.0
18300841 18570191.0
10220077 10538513.0
8867323 9208784.0
178591 578543.0
7444159 7570776.0
295301 431739.0
8644717 8872324.0
1348901 1408681.0
1238220 1333432.0
1490355 1800240.0
3111097 3331459.0
1237568 1481299.0
4612269 4944755.0
56018056 557567

567386 710774.0
4632441 4887138.0
58785 197515.0
1886430 1957733.0
8404590 8617067.0
9402420 9565135.0
10138232 10318174.0
183278 309218.0
365435 612186.0
11152 213791.0
2569098 2622729.0
879377 961588.0
79524 133283.0
155520 256947.0
147959 246980.0
270689 323457.0
5049877 5121950.0
3310815 3473693.0
17330424 17641115.0
270559 365610.0
111417 184153.0
134276 192153.0
325101 427026.0
3223450 3296328.0
15858359 15783773.0
783788 846625.0
5873903 5936358.0
36593 154050.0
1501658 1564384.0
23 584932.0
560529 15948.0
14 825548.0
789557 24733.0
155830 217413.0
4820764 4883459.0
3055370 3113921.0
7384073 7604589.0
3175218 3354388.0
137951697 138215060.0
557302 729271.0
67890 153733.0
2676984 2790064.0
2099056 2245554.0
1706036 1824478.0
143906 224587.0
1895121 1993568.0
1124398 1271567.0
3449294 3691045.0
875057 1063562.0
1851065 1920511.0
6227970 6354125.0
4857825 4991458.0
190841 266261.0
4585897 4655159.0
6012154 6095429.0
5278884 5393899.0
4211600 4391158.0
453029159 16761354.0
16730529 

11278031 11489884.0
14858340 15049910.0
1132886 1314655.0
3750749 3948020.0
5209441 5461102.0
4014347 4223349.0
16917988 17243132.0
8656132 9090148.0
7575790 7793379.0
4382626 4466405.0
9817190 10051526.0
10042885 10233368.0
16795455 17009594.0
69256871 69445721.0
20865863 21105684.0
5804942 6084491.0
5784703 5952236.0
12082803 12278514.0
2033756566 2033958237.0
5533734 5709032.0
11769199 11938201.0
15387285 15601645.0
15234387 15451414.0
5156230 5376737.0
14105917 14319495.0
651831 759624.0
2680728 2789530.0
6119088 6232883.0
7024107 7152483.0
1230519 1425949.0
8426594 8723427.0
22778226 23195521.0
4552585 4661991.0
1482354 1636474.0
643064 724441.0
1672111 1898491.0
1280559774 1280954258.0
241291922 241717716.0
4386754 4903629.0
44548000 44166917.0
1555638408 1555281269.0
1416112091 1416598888.0
30327796 30806984.0
413323017 412879434.0
528628 637859.0
9292789 9634736.0
3754226 3998646.0
2634958 2818109.0
2637638 2802745.0
2367961 2503759.0
1801041 1902878.0
2050106 2177886.0
3299646

In [759]:
# In step 17, we added a new column...lets look at it
turnstiles_daily.head()

,STATION_LINE,UNIT,C/A,SCP,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES,DAILY_ENTRIES
1,1 AV L,R248,H007,00-00-00,2019-10-27,15101784,2019-10-26,"15,099,179.0","2,605.0"
2,1 AV L,R248,H007,00-00-00,2019-10-28,15102950,2019-10-27,"15,101,784.0","1,166.0"
3,1 AV L,R248,H007,00-00-00,2019-10-29,15106496,2019-10-28,"15,102,950.0","3,546.0"
4,1 AV L,R248,H007,00-00-00,2019-10-30,15110479,2019-10-29,"15,106,496.0","3,983.0"
5,1 AV L,R248,H007,00-00-00,2019-10-31,15114428,2019-10-30,"15,110,479.0","3,949.0"


In [803]:
# 18. Here we sum up the daily_entries per station
final_daily_df = turnstiles_daily[['STATION_LINE','UNIT','C/A','SCP','DATE','DAILY_ENTRIES']]

final_daily_df

,STATION_LINE,UNIT,C/A,SCP,DATE,DAILY_ENTRIES
1,1 AV L,R248,H007,00-00-00,01/02/2020,"1,032.0"
2,1 AV L,R248,H007,00-00-00,01/03/2020,"3,375.0"
3,1 AV L,R248,H007,00-00-00,01/04/2020,"3,486.0"
4,1 AV L,R248,H007,00-00-00,01/05/2020,"2,160.0"
5,1 AV L,R248,H007,00-00-00,01/06/2020,"1,602.0"
...,...,...,...,...,...,...
616601,ZEREGA AV 6,R326,R419,00-05-01,12/27/2019,0.0
616602,ZEREGA AV 6,R326,R419,00-05-01,12/28/2019,0.0
616603,ZEREGA AV 6,R326,R419,00-05-01,12/29/2019,0.0
616604,ZEREGA AV 6,R326,R419,00-05-01,12/30/2019,0.0


In [804]:
# 19. We then get top 10 stations over the 4 month period by aggregating all the daily entries together
# For example Grand Central has 14.7M entries 
top10_stations_volume = (final_daily_df.groupby(['STATION_LINE'])['DAILY_ENTRIES'].sum()
                   .reset_index()
                   .sort_values(['DAILY_ENTRIES'],ascending=[False]).head(10))

top10_stations_volume

,STATION_LINE,DAILY_ENTRIES
434,TIMES SQ 42 ST-1237ACENQRSW,"21,053,126.0"
88,34 ST-PENN STA 123ACE,"17,271,017.0"
300,GRD CNTRL-42 ST 4567S,"14,700,396.0"
86,34 ST-HERALD SQ BDFMNQRW,"13,355,811.0"
26,14 ST-UNION SQ 456LNQRW,"10,543,802.0"
291,FULTON ST 2345ACJZ,"9,294,009.0"
115,59 ST COLUMBUS 1ABCD,"7,717,758.0"
99,47-50 STS ROCK BDFM,"6,870,765.0"
391,PATH NEW WTC 1,"6,082,794.0"
114,59 ST 456NQRW,"5,804,739.0"


In [806]:
# 20. Get mean by all columns

mean_all_columns = (final_daily_df.groupby(['STATION_LINE','UNIT','C/A','SCP'])['DAILY_ENTRIES'].mean()
                   .reset_index()
                   .sort_values(['DAILY_ENTRIES'],ascending=[False]))
top10_stations_mean

,STATION_LINE,UNIT,C/A,SCP,DAILY_ENTRIES
3283,GRD CNTRL-42 ST 4567S,R046,R238A,02-03-00,"6,398.032"
3280,GRD CNTRL-42 ST 4567S,R046,R238A,02-00-02,"6,203.96"
3281,GRD CNTRL-42 ST 4567S,R046,R238A,02-00-03,"6,001.096"
1394,59 ST COLUMBUS 1ABCD,R084,N051,02-00-00,"5,833.856"
3285,GRD CNTRL-42 ST 4567S,R046,R238A,02-03-02,"5,782.44"
...,...,...,...,...,...
3686,KINGS HWY BQ,R211,B023,01-05-01,0.0
4741,VERNON-JACKSON 7,R276,R506,02-05-01,0.0
3695,KINGS HWY BQ,R211,B024,00-05-01,0.0
3706,KINGS HWY F,R130,N558,01-05-01,0.0


In [763]:
# 21. To get the average daily entry for each station we can simply divide by 120 days for a rough estimation
#     On the other hand we can use the mean by all columns from the previous step and sum it up
#     As seen from the returned dataframe, there are 463 rows (thats how many stations we have!)
#     Note: we had to combine a few stations together, for instance we trated PENN Station as one station
#     but in the dataset is was broken down into three names.
#     We now have average Daily_ENTRIES per station.
top10_stations_m= (mean_all_columns.groupby(['STATION_LINE'])['DAILY_ENTRIES'].sum().round()
                      .reset_index().sort_values(['DAILY_ENTRIES'],ascending=[False]))

top10_stations_m.head(20)

,STATION_LINE,DAILY_ENTRIES
435,TIMES SQ 42 ST-1237ACENQRSW,"139,037.0"
88,34 ST-PENN STA 123ACE,"137,384.0"
301,GRD CNTRL-42 ST 4567S,"118,001.0"
86,34 ST-HERALD SQ BDFMNQRW,"106,674.0"
26,14 ST-UNION SQ 456LNQRW,"83,689.0"
292,FULTON ST 2345ACJZ,"71,111.0"
116,59 ST COLUMBUS 1ABCD,"61,871.0"
100,47-50 STS ROCK BDFM,"54,167.0"
279,FLUSHING-MAIN 7,"44,797.0"
115,59 ST 456NQRW,"44,694.0"


In [807]:
# 22. In the pervious step we we got the mean daily entries, now we have the entries by date.
top10mask = (turnstiles_daily["STATION_LINE"].isin(['TIMES SQ 42 ST-1237ACENQRSW',
                                                    '34 ST-PENN STA 123ACE',
                                                    'GRD CNTRL-42 ST 4567S',
                                                    '34 ST-HERALD SQ BDFMNQRW',
                                                    '14 ST-UNION SQ 456LNQRW',
                                                    'FULTON ST 2345ACJZ',
                                                    '59 ST COLUMBUS 1ABCD',
                                                    '47-50 STS ROCK BDFM',
                                                    'PATH NEW WTC 1',
                                                    '59 ST 456NQRW']))

df_top10mask = (turnstiles_daily[top10mask][['STATION_LINE','DATE','DAILY_ENTRIES']]
                      .groupby(['STATION_LINE','DATE'])['DAILY_ENTRIES'].sum().round()
                      .reset_index().sort_values(['STATION_LINE','DATE'],ascending=[False,True]))

df_top10mask

,STATION_LINE,DATE,DAILY_ENTRIES
1125,TIMES SQ 42 ST-1237ACENQRSW,01/02/2020,"105,448.0"
1126,TIMES SQ 42 ST-1237ACENQRSW,01/03/2020,"196,558.0"
1127,TIMES SQ 42 ST-1237ACENQRSW,01/04/2020,"189,710.0"
1128,TIMES SQ 42 ST-1237ACENQRSW,01/05/2020,"113,138.0"
1129,TIMES SQ 42 ST-1237ACENQRSW,01/06/2020,"87,812.0"
...,...,...,...
120,14 ST-UNION SQ 456LNQRW,12/27/2019,"62,140.0"
121,14 ST-UNION SQ 456LNQRW,12/28/2019,"71,783.0"
122,14 ST-UNION SQ 456LNQRW,12/29/2019,"57,403.0"
123,14 ST-UNION SQ 456LNQRW,12/30/2019,"44,631.0"
